Seleccionamos la ruta principal de trabajo

In [1]:
import os
from pathlib import Path

root_directory = Path("C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main")


In [2]:
os.chdir(root_directory)
print(root_directory)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main


## Imports


In [3]:
from typing import Any

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

from anomalib import TaskType
from anomalib.data import MVTec
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import Padim, Patchcore, Fastflow, Cfa

In [4]:
datamodule = MVTec(num_workers=0)
datamodule = MVTec(category="bottle")
datamodule.prepare_data() 
datamodule.setup()  

i, data = next(enumerate(datamodule.val_dataloader()))
print(data.keys())

dict_keys(['image_path', 'label', 'image', 'mask'])


In [5]:
model = Cfa(backbone="resnet18")
datamodule2 = MVTec(num_workers=5)

C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torch\overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torch\overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  t

In [6]:
Cfa??

Init signature:
Cfa(
    backbone: str = 'wide_resnet50_2',
    gamma_c: int = 1,
    gamma_d: int = 1,
    num_nearest_neighbors: int = 3,
    num_hard_negative_features: int = 3,
    radius: float = 1e-05,
) -> None
Source:        
class Cfa(AnomalyModule):
    """CFA: Coupled-hypersphere-based Feature Adaptation for Target-Oriented Anomaly Localization.

    Args:
        backbone (str): Backbone CNN network
            Defaults to ``"wide_resnet50_2"``.
        gamma_c (int, optional): gamma_c value from the paper.
            Defaults to ``1``.
        gamma_d (int, optional): gamma_d value from the paper.
            Defaults to ``1``.
        num_nearest_neighbors (int): Number of nearest neighbors.
            Defaults to ``3``.
        num_hard_negative_features (int): Number of hard negative features.
            Defaults to ``3``.
        radius (float): Radius of the hypersphere to search the soft boundary.
            Defaults to ``1e-5``.
    """

    def __init__(
      

In [7]:
callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="pixel_AUROC",
    ),
    EarlyStopping(
        monitor="pixel_AUROC",
        mode="max",
        patience=5,
    ),
]

In [8]:
engine = Engine(
    callbacks=callbacks,
    pixel_metrics="AUROC",
    accelerator="auto",  
    devices=1,
    logger=False,
)
engine.fit(model=model, datamodule=datamodule2)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
No implementation of `configure_transforms` was provided in the Lightning model. Using default transforms from the base class. This may not be suitable for your use case. Please override `configure_transforms` in your model.
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\loops\utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | model                 | CfaModel                 | 3.2 M 
1 | loss                  | CfaLoss                  | 0     
2 | _transform            | Compose       

Training: |          | 0/? [00:00<?, ?it/s]


100%|██████████| 2/2 [00:14<00:00,  7.14s/it]


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

## Validation


In [9]:
test_results = engine.test(
    model=model,
    datamodule=datamodule2,
    ckpt_path=engine.trainer.checkpoint_callback.best_model_path,
)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
Restoring states from the checkpoint path at C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Cfa\MVTec\bottle\v3\weights\lightning\model.ckpt
Loaded model weights from the checkpoint at C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Cfa\MVTec\bottle\v3\weights\lightning\model.ckpt
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.7283116579055786     │
│       image_F1Score       │    0.9927536249160767     │
│        pixel_AUROC        │    0.3618353009223938     │
└───────────────────────────┴───────────────────────────┘

## OpenVINO Inference

Now that we trained and tested a model, we could check a single inference result using OpenVINO inferencer object. This will demonstrate how a trained model could be used for inference.


Before we can use OpenVINO inference, let's export the model to OpenVINO format first.


In [10]:
engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
)

C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torch\onnx\utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_graph_shape_type_inference(
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torch\onnx\utils.py:1209: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_graph_shape_type_inference(


WindowsPath('C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/results/Cfa/MVTec/bottle/latest/weights/openvino/model.xml')

### Load the OpenVINO Model

By default, the output files are saved into `results` directory. Let's check where the OpenVINO model is stored.


In [12]:
output_path = Path(engine.trainer.default_root_dir)
print(output_path)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Cfa\MVTec\bottle\latest


In [13]:
openvino_model_path = output_path / "weights" / "openvino" / "model.bin"
metadata = output_path / "weights" / "openvino" / "metadata.json"
print(openvino_model_path.exists(), metadata.exists())

True True


In [14]:
inferencer = OpenVINOInferencer(
    path=openvino_model_path,  # Path to the OpenVINO IR model.
    metadata=metadata,  # Path to the metadata file.
    device="CPU",  # We would like to run it on an Intel CPU.
)

In [15]:
print(openvino_model_path)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Cfa\MVTec\bottle\latest\weights\openvino\model.bin


In [16]:
import os

# Define la ruta al  que contiene las imágenes
directory = "C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/datasets/MVTec/bottle/test/broken_large/"

# Vamos los archivos del directorio
files = os.listdir(directory)

# Nos quedamos solo con las imagenes 
image_files = [f for f in files if f.endswith('.png')]

# Ahora tenemos un bucle para hacer todas las iteraciones seguidas
for image_file in image_files:

    image_path = os.path.join(directory, image_file)
    predictions = inferencer.predict(image=image_path)
    print(f"Archivo: {image_file}, Score: {predictions.pred_score}, Label: {predictions.pred_label}")


Archivo: 000.png, Score: 0.6440988691477895, Label: 1
Archivo: 001.png, Score: 0.6469823909596405, Label: 1
Archivo: 002.png, Score: 0.5064063985546363, Label: 1
Archivo: 003.png, Score: 0.5484773891022212, Label: 1
Archivo: 004.png, Score: 0.5582008245289322, Label: 1
Archivo: 005.png, Score: 0.546694157403024, Label: 1
Archivo: 006.png, Score: 0.572235223743205, Label: 1
Archivo: 007.png, Score: 0.550967413274654, Label: 1
Archivo: 008.png, Score: 0.6448578984221599, Label: 1
Archivo: 009.png, Score: 0.6127737108388412, Label: 1
Archivo: 010.png, Score: 0.5446048533216133, Label: 1
Archivo: 011.png, Score: 0.5362693463417805, Label: 1
Archivo: 012.png, Score: 0.5784076484586738, Label: 1
Archivo: 013.png, Score: 0.6625272063610734, Label: 1
Archivo: 014.png, Score: 0.5663734904196961, Label: 1
Archivo: 015.png, Score: 0.5634928239956885, Label: 1
Archivo: 016.png, Score: 0.538570956907547, Label: 1
Archivo: 017.png, Score: 0.6390484858817815, Label: 1
Archivo: 018.png, Score: 0.55021

In [17]:
import os

# Define la ruta al directorio que contiene las imágenes
directory = "C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/datasets/MVTec/bottle/test/good/"

# Vamos los archivos del directorio
files = os.listdir(directory)
# Nos quedamos solo con las imagenes 
image_files = [f for f in files if f.endswith('.png')]

# Ahora tenemos un bucle para hacer todas las iteraciones seguidas
for image_file in image_files:

    image_path = os.path.join(directory, image_file)
    predictions = inferencer.predict(image=image_path)
    print(f"Archivo: {image_file}, Score: {predictions.pred_score}, Label: {predictions.pred_label}")

Archivo: 000.png, Score: 0.49859300764016756, Label: 0
Archivo: 001.png, Score: 0.5310540224368189, Label: 1
Archivo: 002.png, Score: 0.4923846386306702, Label: 0
Archivo: 003.png, Score: 0.5031163199030831, Label: 1
Archivo: 004.png, Score: 0.4948790718578611, Label: 0
Archivo: 005.png, Score: 0.82558001404032, Label: 1
Archivo: 006.png, Score: 0.8460621309509628, Label: 1
